In [1]:
import asyncio
from hfc.fabric import Client
import nest_asyncio
nest_asyncio.apply()

In [2]:
loop = asyncio.get_event_loop()

cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user('org1.example.com', 'Admin')

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


In [3]:
# Make the client know there is a channel in the network
cli.new_channel('assetchannel')

New channel with name = assetchannel


In [4]:
# Install Example Chaincode to Peers
# GOPATH setting is only needed to use the example chaincode inside sdk
import os
gopath_bak = os.environ.get('GOPATH', '')
gopath = os.path.normpath(os.path.join(
                      os.path.dirname(os.path.realpath('__file__')),
                      'test/fixtures/chaincode'
                     ))
os.environ['GOPATH'] = os.path.abspath(gopath)
# os.environ['GOPATH'] = os.path.abspath('/home/prixa-ml/go')

In [5]:
# The response should be true if succeed
responses = loop.run_until_complete(cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/atcc',
               cc_name='atcc',
               cc_version='v1.0'
               ))

responses

[response {
   status: 500
   message: "error installing chaincode code atcc:v1.0(chaincode /var/hyperledger/production/chaincodes/atcc.v1.0 exists)"
 }
 payload: "\n \377\007\346\336\346\3603\236\317?\220\250\000\244-\245\237 L>ZH\353eTs\032\017\234\313r\362\022{\032q\010\364\003\022lerror installing chaincode code atcc:v1.0(chaincode /var/hyperledger/production/chaincodes/atcc.v1.0 exists)\"\006\022\004lscc",
 response {
   status: 500
   message: "error installing chaincode code atcc:v1.0(chaincode /var/hyperledger/production/chaincodes/atcc.v1.0 exists)"
 }
 payload: "\n \377\007\346\336\346\3603\236\317?\220\250\000\244-\245\237 L>ZH\353eTs\032\017\234\313r\362\022{\032q\010\364\003\022lerror installing chaincode code atcc:v1.0(chaincode /var/hyperledger/production/chaincodes/atcc.v1.0 exists)\"\006\022\004lscc"]

In [6]:
# Instantiate Chaincode in Channel, the response should be true if succeed
args = ['ksejfbglvikjierustgoiswe','machine-learning', 'ravi-pra', 'ravi@gmail.com']

In [7]:
policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0},
        ]
    }
}

In [8]:
response = loop.run_until_complete(cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='atcc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               ))

print(response)

x is ------------>>> response {
  status: 500
  message: "chaincode with name \'atcc\' already exists"
}
payload: "\n \213GK\351\225\2740\371]\302\244\247\345\030\376z\0069\303\275\252\204&X\205.\n\010:\320\226\230\022P\n\026\022\024\n\004lscc\022\014\n\n\n\004atcc\022\002\010\001\032.\010\364\003\022)chaincode with name \'atcc\' already exists\"\006\022\004lscc"



RuntimeError: chaincode with name 'atcc' already exists

In [ ]:
response

In [ ]:
args = ['a', 'b', '100']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='atcc',
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger, default is on tx event
               #cc_pattern='^invoked*' # if you want to wait for chaincode event and you have a `stub.SetEvent("invoked", value)` in your chaincode
               ))


In [ ]:
response

In [9]:
args = ['b']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_query(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='atcc'
               ))
response

'[{"ID":"asset1","color":"blue","size":5,"owner":"Tomoko","appraisedValue":300},{"ID":"asset2","color":"red","size":5,"owner":"Brad","appraisedValue":400},{"ID":"asset3","color":"green","size":10,"owner":"Jin Soo","appraisedValue":500},{"ID":"asset4","color":"yellow","size":10,"owner":"Max","appraisedValue":600},{"ID":"asset5","color":"black","size":15,"owner":"Adriana","appraisedValue":700},{"ID":"asset6","color":"white","size":15,"owner":"Michel","appraisedValue":800}]'

In [9]:
response = loop.run_until_complete(cli.chaincode_assetexists(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=['asset2'],
               cc_name='atcc'
               ))
response

'true'

In [10]:
response = loop.run_until_complete(cli.chaincode_readasset(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=['asset4'],
               cc_name='atcc'
               ))
response

'{"ID":"asset4","color":"yellow","size":10,"owner":"Max","appraisedValue":600}'

In [11]:
response = loop.run_until_complete(cli.chaincode_assetdelete(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=['asset2'],
               cc_name='atcc'
               ))
response

Deleting asset asset2...
res is ------->> [version: 1
response {
  status: 200
}
payload: "\n m\\\326Z\265\\!\001\236Q\372\313\364\300\247|\223y\200\024\234\254r#\"\202t\026\273E\021\317\022O\n:\022\"\n\004atcc\022\032\n\014\n\006asset2\022\002\010\001\032\n\n\006asset2\020\001\022\024\n\004lscc\022\014\n\n\n\004atcc\022\002\010\001\032\003\010\310\001\"\014\022\004atcc\032\004v1.0"
endorsement {
  endorser: "\n\007Org1MSP\022\252\006-----BEGIN CERTIFICATE-----\nMIICKDCCAc+gAwIBAgIRAK5JdC2UorbI3GYUD7Z+I08wCgYIKoZIzj0EAwIwczEL\nMAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3JuaWExFjAUBgNVBAcTDVNhbiBG\ncmFuY2lzY28xGTAXBgNVBAoTEG9yZzEuZXhhbXBsZS5jb20xHDAaBgNVBAMTE2Nh\nLm9yZzEuZXhhbXBsZS5jb20wHhcNMTgxMDE5MDM0ODAwWhcNMjgxMDE2MDM0ODAw\nWjBqMQswCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMN\nU2FuIEZyYW5jaXNjbzENMAsGA1UECxMEcGVlcjEfMB0GA1UEAxMWcGVlcjAub3Jn\nMS5leGFtcGxlLmNvbTBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABF9ACNtWdD+C\nj5hejCA9qvYEO1pbizsTZzpixLZO96Hv7Bhjke0I4m92+m1QB3j3atPbIqYR1vI8\n5PsIw

''

In [14]:
response = loop.run_until_complete(cli.chaincode_assetexists(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=['asset8'],
               cc_name='atcc'
               ))
response

'false'

In [8]:
response = loop.run_until_complete(cli.chaincode_query(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               args=[],
               cc_name='atcc'
               ))
response

'[{"ID":"asset1","color":"blue","size":5,"owner":"Tomoko","appraisedValue":300},{"ID":"asset2","color":"red","size":5,"owner":"Brad","appraisedValue":400},{"ID":"asset3","color":"green","size":10,"owner":"Jin Soo","appraisedValue":500},{"ID":"asset4","color":"yellow","size":10,"owner":"Max","appraisedValue":600},{"ID":"asset5","color":"black","size":15,"owner":"Adriana","appraisedValue":700},{"ID":"asset6","color":"white","size":15,"owner":"Michel","appraisedValue":800}]'

In [ ]:
# !go get github.com/hyperledger/fabric-chaincode-go/shim

In [ ]:
# !go get github.com/hyperledger/fabric-protos-go/peer

In [ ]:
responses = loop.run_until_complete(cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/example_cc',
               cc_name='example_cc',
               cc_version='v1.0'
               ))
responses

In [ ]:
args = ['a', '200', 'b', '300']
response = loop.run_until_complete(cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='artchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               ))

In [ ]:
response

In [ ]:
# Invoke a chaincode
args = ['a', 'b', '100']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='artchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger, default is on tx event
               #cc_pattern='^invoked*' # if you want to wait for chaincode event and you have a `stub.SetEvent("invoked", value)` in your chaincode
               ))
response

In [ ]:
args = ['b']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_query(
               requestor=org1_admin,
               channel_name='artchannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc'
               ))

response

In [ ]:
from hfc.fabric_network.gateway import Gateway
from hfc.fabric_network.network import Network
from hfc.fabric_network.contract import Contract

In [ ]:
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin')

new_gateway = Gateway() # Creates a new gateway instance
options = {'wallet': ''}
response = loop.run_until_complete(new_gateway.connect('test/fixtures/network.json', options))

In [ ]:
new_network = loop.run_until_complete(new_gateway.get_network('artchannel', org1_admin))
new_contract = new_network.get_contract('example_cc')

In [ ]:
response = loop.run_until_complete(new_contract.submit_transaction('artchannel', ['a', 'b', '100'], org1_admin))

response

In [ ]:
response  = loop.run_until_complete(new_contract.evaluate_transaction('artchannel', ['b'], org1_admin))
response